# Assignment 3

### Imports And Functions

In [33]:
import matlib
import numpy as np

def illustrateCurvatures(Kx,Ky,Kxy):
    from mpl_toolkits.mplot3d import Axes3D
    import matplotlib.pyplot as plt
    from matplotlib import cm
    from matplotlib.ticker import LinearLocator, FormatStrFormatter
    import numpy as np
    fig, ax = plt.subplots(subplot_kw={"projection": "3d"})
    #ax = fig.gca(projection='3d')
    ax = fig.gca()
    # Make data.
    X = np.arange(-0.5, 0.6, 0.1)
    Y = np.arange(-0.5, 0.6, 0.1)
    X, Y = np.meshgrid(X, Y)
    Z1 =  (- Kx*X**2 - Ky*Y**2 - Kxy*X*Y)/2
    surf1 = ax.plot_surface(X, Y, Z1, cmap=cm.coolwarm,
                        linewidth=0, antialiased=False)

def laminateThickness(layup):
    return np.sum([layer['thi'] for layer in layup])

def S2D(m):
    return np.array([[        1/m['E1'], -m['v12']/m['E1'],          0],
                     [-m['v12']/m['E1'],         1/m['E2'],          0],
                     [                0,                 0, 1/m['G12']]], float)

def T2Ds(rot):
    c,s = np.cos(np.radians(rot)), np.sin(np.radians(rot))
    return np.array([[ c*c ,  s*s ,   2*c*s],
                     [ s*s ,  c*c ,  -2*c*s],
                     [-c*s,   c*s , c*c-s*s]], float)

def T2De(rot):
    c,s = np.cos(np.radians(rot)), np.sin(np.radians(rot))
    return np.array([[   c*c,   s*s,     c*s ],
                     [   s*s,   c*c,    -c*s ],
                     [-2*c*s, 2*c*s, c*c-s*s ]], float)

def Q2D(m,rot=0):
    S=S2D(m)
    Q=np.linalg.inv(S)
    if rot==0:
        return Q
    else:
        return np.dot(np.linalg.inv( T2Ds(rot) ) , np.dot(Q,T2De(rot)) )

def laminateStiffnessMatrix(layup):
    ABD=np.zeros((6,6),float)
    hbot = -laminateThickness(layup)/2    
    for layer in layup:
        htop = hbot + layer['thi'] 
        Qt = Q2D( layer['mat'], layer['ori'])
        ABD[0:3,0:3] = ABD[0:3,0:3] +       Qt*(htop   -hbot   )
        ABD[0:3,3:6] = ABD[0:3,3:6] + (1/2)*Qt*(htop**2-hbot**2)
        ABD[3:6,0:3] = ABD[3:6,0:3] + (1/2)*Qt*(htop**2-hbot**2)
        ABD[3:6,3:6] = ABD[3:6,3:6] + (1/3)*Qt*(htop**3-hbot**3)
        hbot=htop   
    return ABD

# Layup 1

In [54]:
m1 = matlib.get('Carbon/Epoxy(a)')

layup1 = [
        {'mat': m1, 'ori':  0.0, 'thi': 1.0},
        {'mat': m1, 'ori':-45.0, 'thi': 2.0},
        {'mat': m1, 'ori':  0.0, 'thi': 1.0},
        {'mat': m1, 'ori': 45.0, 'thi': 2.0},
        {'mat': m1, 'ori':  0.0, 'thi': 1.0},
]

### 3 Point Bending Test

In [52]:
def get_bendingStiffness(layup, b=1):
    ADB = laminateStiffnessMatrix(layup)
    Dxs = ADB[3, 5]
    Dys = ADB[4, 5]
    if Dxs != 0 or Dys != 0:
        raise Exception("Dxs or Dys != 0")
    return ADB[3,3] - (ADB[3,4]**2)/ADB[4,4]

def timoshenkoDeformation(F, L, layup, b=1):
    bendingStiffness = get_bendingStiffness(layup)
    w_kir = (F*L**3)/(48*bendingStiffness)
    K = 0.8333333333333
    m = layup[0]['mat']
    G=(3*m['G12']+2*m['G23'])/6
    h = laminateThickness(layup)
    w_tim = (F*L)/(4*K*G*b*h)
    return w_kir + w_tim

w = timoshenkoDeformation(F=100000.0, L=10.0, layup=layup1, b=1)

print('w = {:.3f} mm'.format(w))


w = 13.354 mm


# Tsai-Wu top